In [ ]:
%load_ext jupyter_black

In [ ]:
from datetime import datetime, timedelta
import glob
import os
import re

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [ ]:
print("this is a test")

In [ ]:
df = pd.DataFrame()

path_to_json = "../../runs/"

json_pattern = os.path.join(path_to_json, "*.json")
file_list = glob.glob(json_pattern)

for file in file_list:
    data = pd.read_json(file)
    df = pd.concat([df, data], ignore_index=True)

In [ ]:
df["departure_date_dt"] = pd.to_datetime(
    df["departure_date"].apply(lambda x: f"{x} 2024")
)
df["return_date_dt"] = pd.to_datetime(df["return_date"].apply(lambda x: f"{x} 2024"))

df["departure_dt"] = pd.to_datetime(
    df.apply(lambda row: f"{row['departure']} {row['departure_date']} 2024", axis=1)
)
df["flight_duration_dt"] = pd.to_timedelta(df["duration"])
df["landing_origin_dt"] = df["departure_dt"] + df["flight_duration_dt"]

In [ ]:
df["trip_duration"] = df["return_date_dt"] - df["departure_date_dt"]
df["trip_duration_days"] = df["trip_duration"].dt.days

In [ ]:
df["connection"] = df.apply(
    lambda row: f"{row['origin_airport_code']} - {row['destination_airport_code']}",
    axis=1,
)

In [ ]:
df["stops_str"] = df["stops"].apply(lambda x: x if isinstance(x, str) else "No")

In [ ]:
def parse_duration(duration):
    # Extract hours and minutes using regex
    if type(duration) != str:
        return None
    match = re.match(r"(?:(\d+) hr)?\s*(?:(\d+) min)?\s*(\w{3})$", duration)
    if match:
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0
        return timedelta(hours=hours, minutes=minutes)
    return None


def extract_airport_code(duration):
    # Extract airport code using regex
    if type(duration) != str:
        return None
    match = re.match(r"(?:(\d+) hr)?\s*(?:(\d+) min)?\s*(\w{3})$", duration)
    if match:
        return match.group(3)
    return None


# TODO: This does not handle a case where there are multiple stops on the way.
# The data does not provide information on each of the layovers in that case.
df["layover_duration"] = df["stops"].apply(lambda x: parse_duration(x))
df["layover_airport"] = df["stops"].apply(lambda x: extract_airport_code(x))

In [ ]:
def count_stops(entry):
    if pd.isna(entry):
        return 0
    # Find all 3-letter airport codes
    codes = re.findall(r"\b[A-Z]{3}\b", entry)
    return len(codes)


df["number_of_stops"] = df["stops"].apply(count_stops)

In [ ]:
def parse_landing_time(entry):
    # Use regex to find the time in the format hh:mm AM/PM
    match = re.match(r"(\d{1,2}:\d{2} [APM]{2})", entry)
    if match:
        time_str = match.group(1)
        return datetime.strptime(time_str, "%I:%M %p").time()
    return None


df["landing_dt"] = df["landing"].apply(parse_landing_time)

In [ ]:
df["flight_duration_dt"] = pd.to_timedelta(df["duration"])
df["flight_duration_hours"] = (df["flight_duration_dt"].dt.total_seconds()) / 3600

In [ ]:
# For some searches the median duration will depend on the departure date
# This makes the visualisation properly center around the departure date
duration_mapping = (
    df[["departure_date", "trip_duration_days"]]
    .groupby("departure_date")
    .median()
    .to_dict()["trip_duration_days"]
)

In [ ]:
# Add date jitter for better visualisation

unique_trip_duration_days = df["trip_duration_days"].unique()
trip_duration_median = np.median(unique_trip_duration_days)
df["departure_date_with_jitter"] = df.apply(
    lambda row: row["departure_date_dt"]
    + timedelta(
        hours=2 * (row["trip_duration_days"] - duration_mapping[row["departure_date"]])
    ),
    axis=1,
)

In [ ]:
px.scatter(
    df,
    x="departure_date_with_jitter",
    y="price_unit",
    color="direct_flight",
    opacity=0.5,
)

In [ ]:
# Filter with the agreed rules

# - The duration for non-direct flights should be less than q75 of analysed flights
# - The cost for the flight should be less than q75
# - Number of stops should be less than q75
# - Lower layover bound - 1 hour
# - Upper layover bound is probably included in the total-duration filter
# - Change of airport, as a rule of thumb, is not acceptable
# - arrival time before 8pm local time
# - airlines: no Flair

DURATION_QUANTILE_THRESHOLD = 0.66
COST_QUANTILE_THRESHOLD = 0.75
STOPS_QUANTILE_THRESHOLD = 0.75
MIN_LAYOVER_DURATION = timedelta(hours=1)
LANDING_TIME_MIN = datetime.strptime("04:00 AM", "%I:%M %p").time()
LANDING_TIME_MAX = datetime.strptime("09:00 PM", "%I:%M %p").time()

df_filtered = df.copy()

# The duration for non-direct flights should be less than q75 of analysed flights
non_direct_duration_q75 = df_filtered[~df_filtered["direct_flight"]][
    "flight_duration_hours"
].quantile(DURATION_QUANTILE_THRESHOLD)
df_filtered = df_filtered[
    df_filtered["direct_flight"]
    | (df_filtered["flight_duration_hours"] <= non_direct_duration_q75)
]

# The cost for the flight should be less than q75
cost_q75 = df_filtered["price_unit"].quantile(COST_QUANTILE_THRESHOLD)
df_filtered = df_filtered[df_filtered["price_unit"] <= cost_q75]

# Number of stops should be less than q75
stops_q75 = df_filtered["number_of_stops"].quantile(STOPS_QUANTILE_THRESHOLD)
df_filtered = df_filtered[df_filtered["number_of_stops"] <= stops_q75]

# Lower layover bound - 1 hour
df_filtered = df_filtered[
    df_filtered["layover_duration"].isna()
    | (df_filtered["layover_duration"] >= MIN_LAYOVER_DURATION)
]

# Change of airport, as a rule of thumb, is not acceptable
df_filtered = df_filtered[df_filtered["stops"] != "Change of airport"]

# arrival time during the day
df_filtered = df_filtered[
    (LANDING_TIME_MIN <= df_filtered["landing_dt"])
    & (df_filtered["landing_dt"] <= LANDING_TIME_MAX)
]

In [ ]:
px.violin(df_filtered, x="flight_duration_hours", box=True)

In [ ]:
px.scatter(
    df_filtered,
    y="price_unit",
    x="flight_duration_hours",
    color="direct_flight",
    marginal_y="violin",
    marginal_x="box",
)

In [ ]:
df_filtered.iloc[1]

In [ ]:
fig = px.scatter(
    df_filtered,
    x="departure_date_with_jitter",
    y="price_unit",
    color="direct_flight",
    opacity=0.5,
    custom_data=[
        "connection",
        "departure_dt",
        "landing",
        "duration",
        "stops_str",
        "price_currency",
        "price_unit",
    ],
)

hovertemplate = """
<b>%{customdata[0]}</b><br>
Departure: %{customdata[1]}<br>
Landing: %{customdata[2]}<br>
Duration: %{customdata[3]}<br>
Stops: %{customdata[4]}<br>
Price: %{customdata[5]} %{customdata[6]}
"""

# TODO: add some visualisation for how good the price/duration is relative to others
# e.g. how many % flights are below this duration/prict
# or how does the value compare to extremes (min-max)

fig.update_traces(hovertemplate=hovertemplate)

fig.show()

In [ ]:
df_filtered[df_filtered["price_unit"] < 850][
    [
        "origin_airport_name",
        "destination_airport_name",
        "departure_date",
        "return_date",
        "airline",
    ]
]

In [ ]:
df_sub = df_filtered  # [df_filtered["departure_date_dt"] == datetime(2024, 11, 20)]
line_colors = {True: "#F44336", False: "#2196F3"}

fig = go.Figure()
for _, row in df_sub.iterrows():
    fig.add_trace(
        go.Scatter(
            x=[row["departure_dt"], row["landing_origin_dt"]],
            y=[row["price_unit"], row["price_unit"]],
            line_color=line_colors[row["direct_flight"]],
        )
    )

fig.update_layout(
    autosize=True,
    height=1000,
)

fig.show()